<a href="https://colab.research.google.com/github/IshanMahesh/M3DETR-New-Mod/blob/New-Modifications/New_M3DETR_Ishan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# sanity check hardware
!nvidia-smi

# connect to google drive
from google.colab import drive
drive.mount('/content/drive')

# sanity check required packages are installed
import torch
import torchvision
import pathlib
import shutil

# install M3DETR dependencies
!pip install --default-timeout=100 spconv-cu120 cython

Tue Dec  5 21:02:54 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   49C    P8    12W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
%%time

# clone repository
!git clone https://github.com/danielmohansahu/M3DETR.git
%cd M3DETR

# build repository
!python3 setup.py develop

Cloning into 'M3DETR'...
remote: Enumerating objects: 635, done.
remote: Counting objects: 100% (338/338), done.
remote: Compressing objects: 100% (202/202), done.
remote: Total 635 (delta 169), reused 251 (delta 122), pack-reused 297
Receiving objects: 100% (635/635), 58.89 MiB | 16.26 MiB/s, done.
Resolving deltas: 100% (238/238), done.
/content/M3DETR
running develop
/usr/local/lib/python3.10/dist-packages/setuptools/command/develop.py:40: EasyInstallDeprecationWarning: easy_install command is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` and ``easy_install``.
        Instead, use pypa/build, pypa/installer, pypa/build or
        other standards-based tools.

        See https://github.com/pypa/setuptools/issues/917 for details.
        ********************************************************************************

!!
  easy_install.initialize_options(self)
/usr/local/lib/python3.1

##KITTI Dataset Prep

In [4]:
%%time

# local location of drive cache (mounted)
cache_file = pathlib.Path("/content/drive/MyDrive/cache/m3detr/kitti.zip")

# local root directory of data
data_root_dir = pathlib.Path("/content/M3DETR/data")

# local location of dataset
data_dir = data_root_dir / "kitti"

if not cache_file.exists():
  # download and fully process KITTI from scratch. this will be slow!

  # install dataset management helper
  !python3 -m pip install fiftyone
  !python3 -m pip install fiftyone-db-ubuntu2204

  # import dataset management helper
  import fiftyone

  print("Downloading and processing KITTI dataset...")
  # note - this is kind of a hack to avoid a slow unnecessary operation
  try:
    dataset = fiftyone.zoo.load_zoo_dataset('kitti-multiview', dataset_dir=data_dir.as_posix())
  except ImportError:
    print("Skipping converting PointClouds to PCD; this is unnecessary.")
  else:
    raise RuntimeError("We shouldn't succeed here - you're going to run out of space!")

  # rename folders to follow expected convention
  print("Renaming and removing unneeded files / folders...")
  !cd {data_dir.as_posix()} && mv test testing && mv train training
  !cd {data_dir.as_posix()} && mv testing/left testing/image_2
  !cd {data_dir.as_posix()} && mv training/left training/image_2 && mv training/labels training/label_2

  # remove those we don't care about
  !cd {data_dir.as_posix()} && rm -rf tmp-download testing/right training/right

  # post-process via OpenPCD scripts
  print("Post-processing dataset for OpenPCD...")
  !python3 -m pcdet.datasets.kitti.kitti_dataset create_kitti_infos \
              tools/cfgs/dataset_configs/kitti_dataset.yaml

  # zip up the results for faster copy speeds
  print("Archiving dataset...")
  !cd {data_root_dir.as_posix()} && zip kitti.zip kitti -r

  # finally, copy over results to drive
  print("Backing up to drive...")
  !cd {data_root_dir.as_posix()} && mv kitti.zip {cache_file.as_posix()}

elif not (data_dir / "training").is_dir() or not (data_dir / "testing").is_dir():
  # pull pre-processed KITTI dataset from local drive

  print("Copying over cached KITTI dataset...")
  !cp {cache_file.as_posix()} {data_root_dir.as_posix()}

  print("Unzipping cached KITTI dataset...")
  !cd {data_root_dir.as_posix()} && unzip -o kitti.zip

# sanity check before continuing
for directory in ("calib", "label_2", "image_2", "velodyne"):
  assert (data_dir / "training" / directory).is_dir(), f"Missing required training data : {directory}"
for directory in ("calib", "image_2", "velodyne"):
  assert (data_dir / "testing" / directory).is_dir(), f"Missing required testing data : {directory}"
assert (data_dir / "kitti_dbinfos_train.pkl").is_file(), "Missing required metadata!"
assert (data_dir / "kitti_infos_test.pkl").is_file(), "Missing required metadata!"
assert (data_dir / "kitti_infos_train.pkl").is_file(), "Missing required metadata!"
assert (data_dir / "kitti_infos_trainval.pkl").is_file(), "Missing required metadata!"
assert (data_dir / "kitti_infos_val.pkl").is_file(), "Missing required metadata!"
print("KITTI dataset loaded.")

KITTI dataset loaded.
CPU times: user 2.91 ms, sys: 350 µs, total: 3.26 ms
Wall time: 352 ms


##KITTI Downsampling

In [5]:
import pickle
import os
from time import sleep

# find all processed metadata files (pkl)
pkl = [f for f in data_dir.glob("*.pkl")]
print(f"Trimming each of the following:\n{pkl}")

def trim_f(data):
    """ Trim the given list to 1000 entries. """
    cnt = 0
    l = len(data)
    for i in range(l-1,-1,-1):
        if len(data) >1000:

            del data[len(data)-1]
            # print(cnt,'data entries left')
            cnt = len(data)

        else:
            # print('The data ', cnt, ' is : ', item)
            print('Cutting data ended')
            break

    # sleep(1.5)
    return cnt

for path in pkl:
    with open(path, 'rb+') as f:
        data = pickle.load(f)

    cnt = len(data)

    while( cnt > 1000):

        cnt = trim_f(data)

        if(cnt==1000):
            break

    print('Data downsampling done')

    f.close()

    with open(path, 'wb') as f:
        pickle.dump(data,f)

print(f"Finished downsampling dataset.")

Trimming each of the following:
[PosixPath('/content/M3DETR/data/kitti/kitti_infos_val.pkl'), PosixPath('/content/M3DETR/data/kitti/kitti_dbinfos_train.pkl'), PosixPath('/content/M3DETR/data/kitti/kitti_infos_test.pkl'), PosixPath('/content/M3DETR/data/kitti/kitti_infos_trainval.pkl'), PosixPath('/content/M3DETR/data/kitti/kitti_infos_train.pkl')]
Cutting data ended
Data downsampling done
Data downsampling done
Cutting data ended
Data downsampling done
Cutting data ended
Data downsampling done
Cutting data ended
Data downsampling done
Finished downsampling dataset.


##Evaluation

In [6]:
%%time

# download pre-trained model
!mkdir -p /content/models
!cd /content/models && gdown 1Jwr9keDHVabebtf-ApSs7BH8-RDQIUmj

# change to tools directory going forward
%cd /content/M3DETR/tools

# evaluate
!python3 test.py --cfg_file ./cfgs/kitti_models/M3DETR.yaml --workers 1 \
  --ckpt /content/models/m3detr_kitti2.pth --eval_tag evaluation --batch_size 16

Downloading...
From: https://drive.google.com/uc?id=1Jwr9keDHVabebtf-ApSs7BH8-RDQIUmj
To: /content/models/m3detr_kitti2.pth
100% 174M/174M [00:05<00:00, 33.3MB/s]
/content/M3DETR/tools
2023-12-05 22:56:05,002   INFO  **********************Start logging**********************
2023-12-05 22:56:05,003   INFO  CUDA_VISIBLE_DEVICES=ALL
2023-12-05 22:56:05,003   INFO  cfg_file         ./cfgs/kitti_models/M3DETR.yaml
2023-12-05 22:56:05,003   INFO  batch_size       16
2023-12-05 22:56:05,003   INFO  workers          1
2023-12-05 22:56:05,003   INFO  extra_tag        default
2023-12-05 22:56:05,003   INFO  ckpt             /content/models/m3detr_kitti2.pth
2023-12-05 22:56:05,003   INFO  launcher         none
2023-12-05 22:56:05,003   INFO  tcp_port         18888
2023-12-05 22:56:05,003   INFO  local_rank       0
2023-12-05 22:56:05,003   INFO  set_cfgs         None
2023-12-05 22:56:05,003   INFO  max_waiting_mins 30
2023-12-05 22:56:05,003   INFO  start_epoch      0
2023-12-05 22:56:05,003   I

##Training


In [7]:
%%time

# train
!python3 train.py --cfg_file ./cfgs/kitti_models/M3DETR.yaml --workers 1 \
  --batch_size 4 --epochs 20

# print out total size of resulting checkpoint(s)
%ls -lh ../output/cfgs/kitti_models/M3DETR/default/ckpt/

Streaming output truncated to the last 5000 lines.
train:  42% 106/250 [03:30<04:44,  1.98s/it, total_it=2605]
epochs:  50% 10/20 [1:27:07<1:23:22, 500.30s/it, loss=1.6, lr=0.00903, d_time=0.00(0.01), f_time=1.23(1.22), b_time=1.69(1.69)]
train:  43% 107/250 [03:32<04:43,  1.98s/it, total_it=2606]
epochs:  50% 10/20 [1:27:09<1:23:22, 500.30s/it, loss=1.67, lr=0.00903, d_time=0.00(0.01), f_time=1.22(1.22), b_time=1.68(1.69)]
train:  43% 108/250 [03:34<04:41,  1.98s/it, total_it=2607]
epochs:  50% 10/20 [1:27:11<1:23:22, 500.30s/it, loss=1.51, lr=0.00902, d_time=0.00(0.01), f_time=1.23(1.22), b_time=1.69(1.69)]
train:  44% 109/250 [03:36<04:39,  1.98s/it, total_it=2608]
epochs:  50% 10/20 [1:27:13<1:23:22, 500.30s/it, loss=1.53, lr=0.00902, d_time=0.00(0.01), f_time=1.22(1.22), b_time=1.69(1.69)]
train:  44% 110/250 [03:38<04:36,  1.97s/it, total_it=2609]
epochs:  50% 10/20 [1:27:15<1:23:22, 500.30s/it, loss=1.58, lr=0.00902, d_time=0.00(0.01), f_time=1.20(1.22), b_time=1.65(1.69)]
train